# Download

In [1]:
import os
import base64
import requests
import json
import base64 
import tempfile
from urllib.parse import urlparse
from urllib.parse import unquote

import pandas as pd
import numpy as np
from getpass import getpass
from glob import glob
from tqdm import tqdm
pd.set_option('display.max_columns', None)

access_token = 'ALtAyMZA.Y1sITQvTj6OXSr6XUz31P2ov3XvsdkhpbijDutzY0glCVNohhzM4okEe'

# Neighborhood patterns

## helper function

In [17]:
def safe_file_write(file_path, content):
    # Create a temporary file in the same directory as the target file
    dir_name = os.path.dirname(file_path)
    with tempfile.NamedTemporaryFile(mode='wb', dir=dir_name, delete=False) as temp_file:
        try:
            # Write the content to the temporary file
            temp_file.write(content)
            temp_file.flush()
            os.fsync(temp_file.fileno())
            temp_file.close()

            # Rename the temporary file to the target file (atomic operation)
            os.replace(temp_file.name, file_path)
        except Exception as e:
            # Handle the exception (e.g., log it, raise it, or silently ignore it)
            print(f"An error occurred while writing the file: {e}")
            os.unlink(temp_file.name)
        else:
            print("File successfully written.")

def combine_monthly_pattern_home_panel_summary(data_dir, extend='.csv.gz'):
    print(os.path.join(data_dir, f"*{extend}"))
    all_files = glob(os.path.join(data_dir, f"*{extend}"))

    df = pd.concat([pd.read_csv(f) for f in tqdm(all_files)])

    return df
    
    # print(all_files)

# Merge CSV files
# home_panel_dir = r'F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary'
# home_panel_dir = r'F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary_2023-11-23'
# df = combine_monthly_pattern_home_panel_summary(data_dir=home_panel_dir)
# df.to_csv(r'F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary\Monthly_Patterns_home_panel_summary_2019_2023.csv', index=False)
# df

In [18]:

# Spend patterns:
# url = r'https://app.deweydata.io/external-api/v3/products/eb6e748a-0fdd-4bc7-9dd7-bbed0890948d/files'
# # save_dir = r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'   # Dell 2019
# save_dir = r'D:\SafeGraph\Advan_2023_API\Spend_Patterns'  # Lenova 2018

# POI
url = r'https://app.deweydata.io/external-api/v3/products/63eeb9d8-8439-42b3-a75e-2d9cbdf64cb8/files'
save_dir = r'D:\SafeGraph\Advan_2023_API\POI'  # Lenova 2018


# neighborhood patterns:
# url = r'https://app.deweydata.io/external-api/v3/products/2dfcb598-6e30-49f1-bdba-1deae113a951/files'
# save_dir = r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'   # Dell 2019
# save_dir = r'F:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'  # Lenova 2018

# monthly patterns:
# url = r"https://app.deweydata.io/external-api/v3/products/5acc9f39-1ca6-4535-b3ff-38f6b9baf85e/files"
# save_dir = r'K:\SafeGraph\Advan_2023_API\Monthly_Patterns'

# # weekly patterns:
# url = r"https://app.deweydata.io/external-api/v3/products/176f0262-c1f6-4dbe-be43-6a6eb21bcf8a/files"
# save_dir = r'K:\SafeGraph\Advan_2023_API\Weekly_Patterns'

# # hourly weather:
# url = r"https://app.deweydata.io/external-api/v3/products/a6be0385-8820-4943-a509-6eac4154b4f6/files"

# monthly home panel summary
# url = r'https://app.deweydata.io/external-api/v3/products/8546740c-b0e9-4556-abb9-4bea2cca9ac9/files'
# save_dir = r'F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary'


os.makedirs(save_dir, exist_ok=True)
# set key and API endpoint variables
API_KEY = access_token
PRODUCT_API_PATH = url

In [19]:
# df.to_csv(r'F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary\Monthly_Patterns_home_panel_summary_2019_2023.csv', index=False)

In [20]:
results = requests.get(url=PRODUCT_API_PATH,
                                   params={'page': 1,
                                           # 'partition_key_after': '2019-01-01',   # optionally set date value here
                                           # 'partition_key_before': '2024-01-01', 
                                          }, # optionally set date value here
                                   headers={'X-API-KEY': API_KEY,
                                            'accept': 'application/json'
                                           })
response_json = results.json()
# response_json

In [ ]:
response_json

In [22]:
def get_all_download_urls():
    total_pages = 1
    current_page = 0
    df_list = []
    while current_page < total_pages:
        results = requests.get(url=PRODUCT_API_PATH,
                                   params={'page': total_pages,
                                           # 'partition_key_after': '2017-01-01',   # optionally set date value here
                                           # 'partition_key_before': '2024-12-31', 
                                          }, # optionally set date value here
                                   headers={'X-API-KEY': API_KEY,
                                            'accept': 'application/json'
                                           })
        response_json = results.json()
        total_pages = response_json['total_pages']
    
        df = pd.DataFrame(response_json)
        df_list.append(df)
        current_page += 1
            
    response_df = pd.concat(df_list)
    
    
    df_list2 = []
    for idx, row in response_df.iterrows():
        # print(row['download_links'])
        df = pd.DataFrame.from_dict([row['download_links']])
        df_list2.append(df)
        # break
        
    url_df = pd.concat(df_list2).sort_values("partition_key")
    return response_df, url_df

response_df, url_df = get_all_download_urls()
url_df

,link,partition_key,file_name,file_extension,file_size_bytes,modified_at
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-0.csv.gz,.csv.gz,146193779,2024-01-07T20:48:47.561Z
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-1.csv.gz,.csv.gz,146282352,2024-01-07T20:48:47.556Z
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-2.csv.gz,.csv.gz,154391307,2024-01-07T20:48:47.574Z
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-3.csv.gz,.csv.gz,167781076,2024-01-07T20:48:47.563Z
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-4.csv.gz,.csv.gz,149519890,2024-01-07T20:48:47.566Z
...,...,...,...,...,...,...
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-59.csv.gz,.csv.gz,149649486,2024-01-07T20:48:45.446Z
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-60.csv.gz,.csv.gz,142311650,2024-01-07T20:48:45.436Z
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-61.csv.gz,.csv.gz,127156461,2024-01-07T20:48:45.435Z
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-62.csv.gz,.csv.gz,151423083,2024-01-07T20:48:45.436Z


In [23]:
# url_df['partition_key'] = url_df['modified_at'].str[:10]
# url_df

In [24]:
downloaded_cnt = 0
for partition_key, df in url_df.groupby('partition_key'):
   
    df = df.reset_index()
    for i, row in df.iterrows():
        try:
            # print(partition_key, i + 1)
            # print("Downloading: ", row['link'])
            date = row['partition_key']
            year = date[:4]
            month = date[5:7]
            day = date[-2:]
            file_dir = os.path.join(save_dir, year, month, day)
            os.makedirs(file_dir, exist_ok=True)
            print("Year, month, day:", year, month, day)
    
            # base_name = row['file_name']
            base_name = row['file_name'][0:24] + row['file_name'][-35:-7] + f"_{i + 1}.csv.gz"
            # print("base_name:", base_name)
            
            file_name = os.path.join(file_dir, base_name)
             
            print(f'Downloading file {file_name}')
        
            if os.path.exists(file_name):
                print(f"File exists, skip! The file is: {file_name} \n")
                continue
        
            # # loop through download links and save to your computer
            data = requests.get(row['link'], stream=True)
            safe_file_write(file_path=file_name, content=data.content)
             
            #  += 1
            downloaded_cnt += 1
            print(f"    Downloaded {i + 1} / {len(df)} files for {partition_key}.")
            print(f"Downloaded {downloaded_cnt} / {len(url_df)} files in total.")
            # print(f"Downloaded {download_count} files in total, Page {page} / {total_pages}.")
            
            print()
        except Exception as e:
            print("Error:", e)
            continue

print("Done.")

Done.


In [25]:
# STOP

# Not used below


SyntaxError: invalid syntax (3138096080.py, line 3)

# Extract POI

In [3]:
data_dir = r"D:\SafeGraph\Advan_2023_API\POI\2024\01\07"
save_dir = r'D:\SafeGraph\Advan_2023_API\POI\2024\01\07\South_Carolina'

os.makedirs(save_dir, exist_ok=True)

In [4]:
all_files = glob(os.path.join(data_dir, "*.gz"))
print("File count:", len(all_files))
print(all_files[0])
print(all_files[-1])

File count: 64
D:\SafeGraph\Advan_2023_API\POI\2024\01\07\Global_Places_POI_Data-0Global_Places_POI_Data-0_1.csv.gz
D:\SafeGraph\Advan_2023_API\POI\2024\01\07\Global_Places_POI_Data-9Global_Places_POI_Data-9_10.csv.gz


In [36]:
countries = ['US']

In [45]:
df_list = []
use_cols = ['PLACEKEY', 'TOP_CATEGORY', 'SUB_CATEGORY', 'NAICS_CODE', 'CITY', 'REGION', 'POSTAL_CODE', 'ISO_COUNTRY_CODE']

for idx, f in enumerate(all_files):
    print(f"Processing: {idx + 1} / {len(all_files)}, ", os.path.basename(f))
    df = pd.read_csv(f, usecols=use_cols)
    t_df = df[df['ISO_COUNTRY_CODE'].isin(counties)] 
    # t_df = df[df['NAICS_CODE'].isin(target_NAICS)]
    # t_df = t_df[t_df['CITY'].isin(target_city)]
    # t_df = t_df[t_df['REGION'].isin(target_region)]
    
    df_list.append(t_df)
    # break

df_all = pd.concat(df_list)
fname = os.path.join(save_dir, 'megerd_POI_20240107.csv')
df_all.to_csv(fname, index=False)
# print("Saved at:", fname)
df_all

Processing: 1 / 64,  Global_Places_POI_Data-0Global_Places_POI_Data-0_1.csv.gz
Processing: 2 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-10_11.csv.gz
Processing: 3 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-11_12.csv.gz
Processing: 4 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-12_13.csv.gz
Processing: 5 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-13_14.csv.gz
Processing: 6 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-14_15.csv.gz
Processing: 7 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-15_16.csv.gz
Processing: 8 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-16_17.csv.gz
Processing: 9 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-17_18.csv.gz
Processing: 10 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-18_19.csv.gz
Processing: 11 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-19_20.csv.gz
Processing: 12 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-1_2.csv.gz
Processing: 13 / 64,  Global_

,PLACEKEY,TOP_CATEGORY,SUB_CATEGORY,NAICS_CODE,CITY,REGION,POSTAL_CODE,ISO_COUNTRY_CODE
0,22r-223@5z5-t7y-gc5,Florists,Florists,453110.0,San Marcos,CA,92078,US
2,22p-222@8fv-zyb-fvf,Lessors of Real Estate,Lessors of Residential Buildings and Dwellings,531110.0,Dania Beach,FL,33004,US
3,22b-225@627-s8r-mzf,Offices of Dentists,NaN,6212.0,New York,NY,10021,US
8,22n-222@8ts-zgc-r6k,Other Personal Services,All Other Personal Services,812990.0,Mesa,AZ,85201,US
9,223-223@5p7-pvf-hwk,Couriers and Express Delivery Services,Couriers and Express Delivery Services,492110.0,Waubay,SD,57273,US
...,...,...,...,...,...,...,...,...
1474541,222-223@5vh-9mv-y9z,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,Fremont,CA,94538,US
1474542,237-222@5z4-zd6-gzf,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,North Hollywood,CA,91601,US
1474546,zzy-222@5qc-w82-ffz,Urban Transit Systems,Bus and Other Motor Vehicle Transit Systems,485113.0,Salt Lake City,UT,84123,US
1474547,22j-222@8dg-gtw-jn5,Lessors of Real Estate,Lessors of Residential Buildings and Dwellings,531110.0,Naples,FL,34108,US


In [ ]:
df_all

# Help topic in Deweydata community

https://community.deweydata.io/t/residing-device-count-in-2023-07-and-2023-08-surged-abnomorally/26675